# Benchmarking typed stable code

In [1]:
using BenchmarkTools

INFO: Recompiling stale cache file /Users/davidbuchacaprats/.julia/lib/v0.6/JLD.ji for module JLD.


In [2]:
function sumsqrtn(n::Int)
    r = 0
    for i = 1:n
        r = r + sqrt(i)
    end
return r end

sumsqrtn (generic function with 1 method)

In [3]:
@time sumsqrtn(10)
@time sumsqrtn(10^7)

  0.017507 seconds (2.11 k allocations: 102.550 KiB)
  0.319274 seconds (30.00 M allocations: 457.764 MiB, 8.71% gc time)


2.1081852648716972e10

In [4]:
typeof(sqrt(4)), typeof(0)

(Float64,Int64)

#### what is wrong with the previous code?

Notice that **```r```** changes its type inside the function.
It is first defined as an integer but then it changes to float.


You can check how the compiler infers types of the variables inside a function using the **```@code_warntype```** method.

In [7]:
@code_warntype sumsqrtn(10)

Variables:
  #self#::#sumsqrtn
  n::Int64
  i::Int64
  #temp#@_4::Int64
  r::Any
  #temp#@_6::Core.MethodInstance
  #temp#@_7::Float64

Body:
  begin 
      r::Any = 0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1,n::Int64)::Bool,n::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#@_4::Int64 = 1
      5: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#@_4::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(2),1)))::Bool)) goto 30
      SSAValue(3) = #temp#@_4::Int64
      SSAValue(4) = (Base.box)(Int64,(Base.add_int)(#temp#@_4::Int64,1))
      i::Int64 = SSAValue(3)
      #temp#@_4::Int64 = SSAValue(4) # line 4:
      unless (r::Union{Float64,Int64} isa Float64)::Any goto 15
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Float64, ::Float64)
      goto 24
      15: 
      unless (r::Union{Float64,Int64} isa Int64)::Any goto 19
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Int64, ::Float64)
      goto 24
      19

Let us rewrite the function defining ```r```as float. Doing so we will avoid changing its type inside the function and it will run faster.

In [5]:
function sumsqrtn2(n::Int64) 
    r = 0.
    for i = 1:n
        r = r + sqrt(i)
    end
return r end

sumsqrtn2 (generic function with 1 method)

In [6]:
@time sumsqrtn2(10^3)
@time sumsqrtn2(10^7)

  0.007542 seconds (1.33 k allocations: 64.019 KiB)
  0.074949 seconds (6 allocations: 192 bytes)


2.1081852648716972e10

In [14]:
@code_warntype sumsqrtn2(10)

Variables:
  #self#::#sumsqrtn2
  n::Int64
  i::Int64
  #temp#::Int64
  r::Float64

Body:
  begin 
      r::Float64 = 0.0 # line 3:
      SSAValue(3) = (Base.select_value)((Base.sle_int)(1,n::Int64)::Bool,n::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(3),1)))::Bool)) goto 16
      SSAValue(4) = #temp#::Int64
      SSAValue(5) = (Base.box)(Int64,(Base.add_int)(#temp#::Int64,1))
      i::Int64 = SSAValue(4)
      #temp#::Int64 = SSAValue(5) # line 4:
      SSAValue(2) = (Base.Math.box)(Base.Math.Float64,(Base.Math.sqrt_llvm)((Base.box)(Float64,(Base.sitofp)(Float64,i::Int64))))::Float64
      r::Float64 = (Base.box)(Base.Float64,(Base.add_float)(r::Float64,SSAValue(2)))
      14: 
      goto 5
      16:  # line 6:
      return r::Float64
  end::Float64


### optimize operations

```julia
function sumsqrtn(n::Int)
    r = 0
    for i = 1:n
        r = r + sqrt(i)
    end
return r end
```

In [14]:
function sum_opt2(n) 
    r = 0.
    for i = 1:n
        r = r + sqrt(i)
    end
return r end

sum_opt2 (generic function with 1 method)

In [15]:
@time sumsqrtn(10^7)

  0.315240 seconds (30.00 M allocations: 457.764 MiB, 9.52% gc time)


2.1081852648716972e10

In [12]:
@time sum_opt2(10^7)

  0.074096 seconds (6 allocations: 192 bytes)


2.1081852648716972e10

In [13]:
@code_warntype sum_opt2(10)

Variables:
  #self#::#sum_opt2
  n::Int64
  i::Int64
  #temp#::Int64
  r::Float64

Body:
  begin 
      r::Float64 = 0.0 # line 3:
      SSAValue(3) = (Base.select_value)((Base.sle_int)(1,n::Int64)::Bool,n::Int64,(Base.sub_int)(1,1)::Int64)::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#::Int64 === (Base.add_int)(SSAValue(3),1)::Int64)::Bool)::Bool goto 16
      SSAValue(4) = #temp#::Int64
      SSAValue(5) = (Base.add_int)(#temp#::Int64,1)::Int64
      i::Int64 = SSAValue(4)
      #temp#::Int64 = SSAValue(5) # line 4:
      SSAValue(2) = (Base.Math.sqrt_llvm)((Base.sitofp)(Float64,i::Int64)::Float64)::Float64
      r::Float64 = (Base.add_float)(r::Float64,SSAValue(2))::Float64
      14: 
      goto 5
      16:  # line 6:
      return r::Float64
  end::Float64


In [100]:
# In this case r is Any 
# The type of r changes and this heavily penalizes speed
#@code_warntype sumsqrtn(10);

In [101]:
# In this case r is Float64 during the execution
#@code_warntype sumsqrtn2(10)

## Use ```local``` to fix type of a variable

If we do not want to profile the code using **```@code_warntype```** we can type the **'''local'''** variables to ensure the type of the variables is not done incorrectly.

In [1]:
workspace()

In [14]:
function sumsqrtn(n::Int)
    r::Float64 = 0
    
    for i = 1:n
        r = r + sqrt(i)
    end
return r end

sumsqrtn (generic function with 1 method)

In [15]:
@time sumsqrtn(10^7)

  0.081791 seconds (3.03 k allocations: 130.746 KB)


2.1081852648716972e10

In [16]:
function sumsqrtn(n::Int)
    r::Int64 = 0
    
    for i = 1:n
        r = r + sqrt(i)
    end
return r end

sumsqrtn (generic function with 1 method)

In [17]:
@time sumsqrtn(10^7)

LoadError: InexactError()